In [1]:
DROP TABLE IF EXISTS complete_joint_dataset;
CREATE TEMP TABLE complete_joint_dataset AS
SELECT
  rental.customer_id,
  inventory.film_id,
  film.title,
  rental.rental_date,
  category.name AS category_name
FROM dvd_rentals.rental
INNER JOIN dvd_rentals.inventory
  ON rental.inventory_id = inventory.inventory_id
INNER JOIN dvd_rentals.film
  ON inventory.film_id = film.film_id
INNER JOIN dvd_rentals.film_category
  ON film.film_id = film_category.film_id
INNER JOIN dvd_rentals.category
  ON film_category.category_id = category.category_id;

SELECT * FROM complete_joint_dataset limit 10;

NOTICE: table "complete_joint_dataset" does not exist, skipping

Commands completed successfully

Commands completed successfully

(10 row(s) affected)

Total execution time: 00:00:00.157

customer_id,film_id,title,rental_date,category_name
130,80,BLANKET BEVERLY,2005-05-24T22:53:30,Family
459,333,FREAKY POCUS,2005-05-24T22:54:33,Music
408,373,GRADUATE LORD,2005-05-24T23:03:39,Children
333,535,LOVE SUICIDES,2005-05-24T23:04:41,Horror
222,450,IDOLS SNATCHERS,2005-05-24T23:05:21,Children
549,613,MYSTIC TRUMAN,2005-05-24T23:08:07,Comedy
269,870,SWARM GOLD,2005-05-24T23:11:53,Horror
239,510,LAWLESS VISION,2005-05-24T23:31:46,Animation
126,565,MATRIX SNOWMAN,2005-05-25T00:00:40,Foreign
399,396,HANGING DEEP,2005-05-25T00:02:21,Drama


In [2]:
SELECT 
    customer_id, 
    category_name, 
    COUNT(*) AS rental_count
FROM complete_joint_dataset
WHERE customer_id in (1, 2, 3)
GROUP BY 
    customer_id,
    category_name
ORDER BY 
    customer_id,
    rental_count DESC; 

/* Customer 3: both Sci-Fi and Animation categories have a rental_count 
value of 3 */
-- This is a tie 

(40 row(s) affected)

Total execution time: 00:00:00.060

customer_id,category_name,rental_count
1,Classics,6
1,Comedy,5
1,Drama,4
1,Action,2
1,Music,2
1,New,2
1,Sci-Fi,2
1,Sports,2
1,Animation,2
1,Documentary,1


In [3]:
-- dealing with ties 
/* Creating joint data set with rental_date used as additional sort 
to deal with ties */

DROP TABLE IF EXISTS complete_joint_dataset_rental_date; 
CREATE TEMP TABLE complete_joint_dataset_rental_date AS 
SELECT 
    rental.customer_id, 
    rental.rental_date,
    inventory.film_id, 
    film.title, 
    category.name AS category_name 
FROM dvd_rentals.rental
INNER JOIN dvd_rentals.inventory
    ON rental.inventory_id = inventory.inventory_id
INNER JOIN dvd_rentals.film
    ON inventory.film_id = film.film_id 
INNER JOIN dvd_rentals.film_category
    ON film.film_id = film_category.film_id 
INNER JOIN dvd_rentals.category
    ON film_category.category_id = category.category_id; 

-- Performing group by aggregations on category_name and customer_id
SELECT 
    customer_id, 
    category_name, 
    COUNT(*) AS rental_count, 
    MAX(rental_date) AS latest_rental_date 
FROM complete_joint_dataset_rental_date
-- note the different filter 
WHERE customer_id = 3 
GROUP BY 
    customer_id, 
    category_name
ORDER BY 
    customer_id, 
    rental_count DESC, 
    latest_rental_date DESC 


NOTICE: table "complete_joint_dataset_rental_date" does not exist, skipping

Commands completed successfully

Commands completed successfully

(13 row(s) affected)

Total execution time: 00:00:00.079

customer_id,category_name,rental_count,latest_rental_date
3,Action,4,2005-07-29T11:07:04
3,Sci-Fi,3,2005-08-22T09:37:27
3,Animation,3,2005-08-18T14:49:55
3,Music,2,2005-08-23T07:10:14
3,Comedy,2,2005-08-20T06:14:12
3,Horror,2,2005-07-31T11:32:58
3,Sports,2,2005-07-30T13:31:20
3,New,2,2005-07-28T04:46:30
3,Games,2,2005-07-27T04:54:42
3,Classics,1,2005-08-01T14:19:48


In [4]:
-- top 2 category average 
DROP TABLE IF EXISTS top_2_category_rental_count; 
CREATE TEMP TABLE top_2_category_rental_count AS 
WITH input_data (customer_id, category_name, rental_count) AS (
    VALUES 
    (1, 'Classics', 6),
    (1, 'Comedy', 5),
    (2, 'Sports', 5),
    (2, 'Classics', 4), 
    (3, 'Action', 4),
    (3, 'Sci-fi', 3)
)
SELECT * FROM input_data; 

SELECT * FROM top_2_category_rental_count

NOTICE: table "top_2_category_rental_count" does not exist, skipping

Commands completed successfully

Commands completed successfully

(6 row(s) affected)

Total execution time: 00:00:00.079

customer_id,category_name,rental_count
1,Classics,6
1,Comedy,5
2,Sports,5
2,Classics,4
3,Action,4
3,Sci-fi,3


In [5]:
-- comparing averages for customers 1, 2, 3 
-- aggregated averages 
WITH aggregated_rental_count AS (
    SELECT 
        customer_id,
        category_name, 
        COUNT(*) AS rental_count
    FROM complete_joint_dataset_rental_date
    WHERE customer_id in (1, 2, 3)
    GROUP BY 
        customer_id,
        category_name 
)

SELECT 
    category_name,
    ROUND(AVG(rental_count), 1) AS avg_rental_count
FROM aggregated_rental_count
GROUP BY 
    category_name
ORDER BY 
    category_name; 


-- avg top 2 rental count
SELECT
    category_name,
    ROUND(AVG(rental_count), 1) as average_rental_count
FROM top_2_category_rental_count
GROUP BY category_name 
ORDER BY category_name; 

(16 row(s) affected)

(5 row(s) affected)

Total execution time: 00:00:00.062

category_name,avg_rental_count
Action,3.0
Animation,2.7
Children,1.0
Classics,3.7
Comedy,3.5
Documentary,1.0
Drama,2.5
Family,1.0
Foreign,1.0
Games,1.7


category_name,average_rental_count
Action,4.0
Classics,5.0
Comedy,5.0
Sci-fi,3.0
Sports,5.0


# 3\. Data Aggregation on Whole Dataset

## 3.1. Customer Rental Count

In [4]:
-- Data aggregation on whole dataset

-- 3.1 : customer rental count
DROP TABLE IF EXISTS category_rental_counts; 
CREATE TEMP TABLE category_rental_counts AS 
SELECT
    customer_id, 
    category_name, 
    COUNT(*) AS rental_count,
    MAX(rental_date) AS latest_rental_date
FROM complete_joint_dataset_rental_date
GROUP BY
    customer_id,
    category_name; 

SELECT *
FROM category_rental_counts
WHERE customer_id = 1 
ORDER BY 
    rental_count DESC 


NOTICE: table "category_rental_counts" does not exist, skipping

Commands completed successfully

Commands completed successfully

(14 row(s) affected)

Total execution time: 00:00:00.042

customer_id,category_name,rental_count,latest_rental_date
1,Classics,6,2005-08-19T09:55:16
1,Comedy,5,2005-08-22T19:41:37
1,Drama,4,2005-08-18T03:57:29
1,Sci-Fi,2,2005-08-21T23:33:57
1,Animation,2,2005-08-22T20:03:46
1,Sports,2,2005-07-08T07:33:56
1,Music,2,2005-07-09T16:38:01
1,Action,2,2005-08-17T12:37:54
1,New,2,2005-08-19T13:56:54
1,Travel,1,2005-07-11T10:13:46


## 3.2. Total Customer Rentals

In [5]:
DROP TABLE IF EXISTS total_customer_rentals; 
CREATE TEMP TABLE total_customer_rentals AS 
SELECT 
    customer_id,
    SUM(rental_count) AS total_rental_count
FROM category_rental_counts
GROUP BY
    customer_id; 

SELECT *
FROM total_customer_rentals
WHERE customer_id <= 5
ORDER BY customer_id;


NOTICE: table "total_customer_rentals" does not exist, skipping

Commands completed successfully

Commands completed successfully

(5 row(s) affected)

Total execution time: 00:00:00.023

customer_id,total_rental_count
1,32
2,27
3,26
4,22
5,38


## 3.3. Average Category Rental Counts

In [6]:
DROP TABLE IF EXISTS avg_category_rental_counts; 
CREATE TEMP TABLE avg_category_rental_counts AS 
SELECT
    category_name, 
    AVG(rental_count) AS avg_rental_count
FROM category_rental_counts
GROUP BY
    category_name; 

SELECT *
FROM avg_category_rental_counts
ORDER BY 
avg_rental_count DESC; 

-- Ugly decimals! 
-- You have watched 2.3343 more movies than the customer average for sci-fi (WEIRD)



NOTICE: table "avg_category_rental_counts" does not exist, skipping

Commands completed successfully

Commands completed successfully

(16 row(s) affected)

Total execution time: 00:00:00.018

category_name,avg_rental_count
Animation,2.3320000000000000
Sports,2.2716763005780347
Family,2.1876247504990020
Action,2.1803921568627451
Documentary,2.1739130434782609
Sci-Fi,2.1715976331360947
Drama,2.1157684630738523
Foreign,2.0953346855983773
Games,2.0443037974683544
New,2.0085470085470085


## 3.4. Update Table Values

In [11]:
-- first create a copy of average_category_rental_counts
DROP TABLE IF EXISTS testing_avg_category_rental_count; 
CREATE TEMP TABLE testing_avg_category_rental_count AS
    TABLE avg_category_rental_counts; 

-- now update all the things!
UPDATE testing_avg_category_rental_count
SET 
    avg_rental_count = avg_rental_count + 10,
    category_name = category_name || ' Category'
WHERE 
 -- first character of category_name is 'C'
    LEFT(category_name, 1) = 'C'
-- show all updated rows as the query output
RETURNING *; 

Commands completed successfully

Commands completed successfully

(3 row(s) affected)

Total execution time: 00:00:00.038

category_name,avg_rental_count
Classics Category,12.0064102564102564
Comedy Category,11.9010101010101010
Children Category,11.9605809128630705


In [12]:
-- Inspecting change:GROUP BY
SELECT *
FROM testing_avg_category_rental_count
ORDER BY category_name;

-- changes verified 

(16 row(s) affected)

Total execution time: 00:00:00.125

category_name,avg_rental_count
Action,2.1803921568627451
Animation,2.3320000000000000
Children Category,11.9605809128630705
Classics Category,12.0064102564102564
Comedy Category,11.9010101010101010
Documentary,2.1739130434782609
Drama,2.1157684630738523
Family,2.1876247504990020
Foreign,2.0953346855983773
Games,2.0443037974683544


In [7]:
-- Updating acutal table: avg_category_rental_counts
-- Using FLOOR() to update: An executive decision
UPDATE avg_category_rental_counts
SET avg_rental_count = FLOOR(avg_rental_count)
RETURNING *; 


(16 row(s) affected)

Total execution time: 00:00:00.006

category_name,avg_rental_count
Sports,2
Classics,2
New,2
Family,2
Comedy,1
Animation,2
Travel,1
Music,1
Horror,1
Drama,2


In [14]:
-- Inspecting changes made to avg_category_rental_counts
SELECT * 
FROM avg_category_rental_counts
ORDER BY 
    category_name; 

-- changes verified 

(16 row(s) affected)

Total execution time: 00:00:00.018

category_name,avg_rental_count
Action,2
Animation,2
Children,1
Classics,2
Comedy,1
Documentary,2
Drama,2
Family,2
Foreign,2
Games,2


## 3.5. Percentile Values

In [18]:
SELECT
  customer_id,
  category_name,
  rental_count,
  PERCENT_RANK() OVER (
    PARTITION BY category_name
    ORDER BY rental_count DESC
  ) AS percentile
FROM category_rental_counts
ORDER BY customer_id, rental_count DESC
LIMIT 14;

(14 row(s) affected)

Total execution time: 00:00:00.035

customer_id,category_name,rental_count,percentile
1,Classics,6,0.0021413276231263384
1,Comedy,5,0.006072874493927126
1,Drama,4,0.03
1,Animation,2,0.38877755511022044
1,New,2,0.2676659528907923
1,Action,2,0.33398821218074654
1,Music,2,0.2040358744394619
1,Sports,2,0.34555984555984554
1,Sci-Fi,2,0.30039525691699603
1,Documentary,1,0.6431535269709544


In [15]:
SELECT
  customer_id,
  category_name,
  rental_count,
  -- use ceiling to round up to nearest integer after multiplying by 100
  CEILING(
    100 * PERCENT_RANK() OVER (
      PARTITION BY category_name
      ORDER BY rental_count DESC
    )
  ) AS percentile
FROM category_rental_counts
ORDER BY customer_id, rental_count DESC
LIMIT 2;

/*You’ve watched 6 Classics films, that’s 4 more than the DVD Rental Co 
average and puts you in the top 1% of Classics gurus!*/

(2 row(s) affected)

Total execution time: 00:00:00.064

customer_id,category_name,rental_count,percentile
1,Classics,6,1.0
1,Comedy,5,1.0


In [8]:
-- Creating TEMP TABLE
DROP TABLE IF EXISTS customer_category_percentiles; 
CREATE TEMP TABLE customer_category_percentiles AS 
SELECT
  customer_id,
  category_name,
  rental_count,
  -- use ceiling to round up to nearest integer after multiplying by 100
  CEILING(
    100 * PERCENT_RANK() OVER (
      PARTITION BY category_name
      ORDER BY rental_count DESC
    )
  ) AS percentile
FROM category_rental_counts; 

SELECT * 
FROM customer_category_percentiles
WHERE customer_id = 1
ORDER BY customer_id, percentile 
Limit 2 

NOTICE: table "customer_category_percentiles" does not exist, skipping

Commands completed successfully

Commands completed successfully

(2 row(s) affected)

Total execution time: 00:00:00.047

customer_id,category_name,rental_count,percentile
1,Classics,6,1.0
1,Comedy,5,1.0


# 4\. Using Our Temporary Tables

## 4.1. Joining Temporary Tables

In [9]:
DROP TABLE IF EXISTS customer_category_joint_table; 
CREATE TEMP TABLE customer_category_joint_table AS 
SELECT 
    t1.customer_id,
    t1.category_name, 
    t1.rental_count,
    t2.total_rental_count, 
    t3.avg_rental_count,
    t4.percentile
FROM category_rental_counts AS t1
INNER JOIN total_customer_rentals AS t2
    ON t1.customer_id = t2.customer_id 
INNER JOIN avg_category_rental_counts AS t3
    ON t1.category_name = t3.category_name 
INNER JOIN customer_category_percentiles AS t4
    ON t1.category_name = t4.category_name 
    AND t1.customer_id = t4.customer_id; 


-- inspect customer_id = 1 rows sorted by percentile
SELECT *
FROM customer_category_joint_table
WHERE customer_id = 1
ORDER BY percentile;


NOTICE: table "customer_category_joint_table" does not exist, skipping

Commands completed successfully

Commands completed successfully

(14 row(s) affected)

Total execution time: 00:00:00.082

customer_id,category_name,rental_count,total_rental_count,avg_rental_count,percentile
1,Classics,6,32,2,1.0
1,Comedy,5,32,1,1.0
1,Drama,4,32,2,3.0
1,Music,2,32,1,21.0
1,New,2,32,2,27.0
1,Sci-Fi,2,32,2,31.0
1,Action,2,32,2,34.0
1,Sports,2,32,2,35.0
1,Animation,2,32,2,39.0
1,Travel,1,32,1,58.0


## 4.2. Adding in Calculated Fields

In [10]:
DROP TABLE IF EXISTS customer_category_joint_table; 
CREATE TEMP TABLE customer_category_joint_table AS 
SELECT 
    t1.customer_id,
    t1.category_name, 
    t1.rental_count,
    t1.latest_rental_date,
    t2.total_rental_count, 
    t3.avg_rental_count,
    t4.percentile,
    t1.rental_count - t3.avg_rental_count AS average_comparison,
    -- round to nearest integer for percentage after multiplying by 100
    ROUND(100 * t1.rental_count / t2.total_rental_count) AS category_percentage 
    -- integer floor division avoided since total_rental_count is 'numeric' 
FROM category_rental_counts AS t1
INNER JOIN total_customer_rentals AS t2
    ON t1.customer_id = t2.customer_id 
INNER JOIN avg_category_rental_counts AS t3
    ON t1.category_name = t3.category_name 
INNER JOIN customer_category_percentiles AS t4
    ON t1.category_name = t4.category_name 
    AND t1.customer_id = t4.customer_id; 


-- inspect customer_id = 1 rows sorted by percentile
SELECT *
FROM customer_category_joint_table
WHERE customer_id = 1
ORDER BY percentile;


Commands completed successfully

Commands completed successfully

(14 row(s) affected)

Total execution time: 00:00:00.084

customer_id,category_name,rental_count,latest_rental_date,total_rental_count,avg_rental_count,percentile,average_comparison,category_percentage
1,Classics,6,2005-08-19T09:55:16,32,2,1.0,4,19
1,Comedy,5,2005-08-22T19:41:37,32,1,1.0,4,16
1,Drama,4,2005-08-18T03:57:29,32,2,3.0,2,13
1,Music,2,2005-07-09T16:38:01,32,1,21.0,1,6
1,New,2,2005-08-19T13:56:54,32,2,27.0,0,6
1,Sci-Fi,2,2005-08-21T23:33:57,32,2,31.0,0,6
1,Action,2,2005-08-17T12:37:54,32,2,34.0,0,6
1,Sports,2,2005-07-08T07:33:56,32,2,35.0,0,6
1,Animation,2,2005-08-22T20:03:46,32,2,39.0,0,6
1,Travel,1,2005-07-11T10:13:46,32,1,58.0,0,3


## 4.3. Checking Data Types Using the information\_schema.columns Table

In [11]:
SELECT 
    table_name, 
    column_name, 
    data_type
FROM information_schema.columns
WHERE table_name in ('category_rental_counts', 'total_customer_rentals')

(6 row(s) affected)

Total execution time: 00:00:00.022

table_name,column_name,data_type
category_rental_counts,customer_id,smallint
category_rental_counts,category_name,character varying
category_rental_counts,rental_count,bigint
category_rental_counts,latest_rental_date,timestamp without time zone
total_customer_rentals,customer_id,smallint
total_customer_rentals,total_rental_count,numeric


In [ ]:
SELECT 
    schema_name, 
    column_name, 
    column_name
FROM information_schema.columns
WHERE column_name LIKE '%rental_count%'

# 5\. Ordering and Filtering Rows with ROW\_NUMBER

In [12]:
DROP TABLE IF EXISTS top_categories_info; 
CREATE TEMP TABLE top_categories_info AS (

WITH ordered_number_category_joint_table AS (
    SELECT
        customer_id, 
        ROW_NUMBER() OVER (
            PARTITION BY customer_id
            ORDER BY rental_count DESC, latest_rental_date DESC
        ) AS category_ranking,
        category_name, 
        rental_count, 
        average_comparison, 
        percentile, 
        category_percentage
    FROM customer_category_joint_table
)
-- filter out top 2 rows from the CTE for final output
SELECT *
FROM ordered_number_category_joint_table
WHERE category_ranking <= 2 
);


NOTICE: table "top_categories_info" does not exist, skipping

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:00.031

In [13]:
SELECT *
FROM top_categories_info
WHERE customer_id in (1, 2, 3)
ORDER BY customer_id, category_ranking;

(6 row(s) affected)

Total execution time: 00:00:00.012

customer_id,category_ranking,category_name,rental_count,average_comparison,percentile,category_percentage
1,1,Classics,6,4,1.0,19
1,2,Comedy,5,4,1.0,16
2,1,Sports,5,3,3.0,19
2,2,Classics,4,2,2.0,15
3,1,Action,4,2,5.0,15
3,2,Sci-Fi,3,1,15.0,12
